<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Using_AutoKeras_for_Automated_Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade autokeras
!pip install tensorflow

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras_tuner import RandomSearch

# Load your dataset
data = pd.read_csv('your_dataset.csv')  # Replace with your dataset path
x = data.drop(columns=['target'])
y = data['target']

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Define the model building function
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(x_train.shape[1],)))
    model.add(keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Initialize the Keras Tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='structured_data'
)

# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters
model = build_model(best_hps)

# Train the model with the optimal hyperparameters
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model
evaluation = model.evaluate(x_test, y_test)
print(f"Model evaluation: {evaluation}")